In [68]:
#todo:implement fashion-mnist:

In [46]:
from zipfile import ZipFile
import os
import sys
import urllib
from model.model import Model
import urllib.request
import numpy as np
import cv2
from layers.layer_dense import Layer_Dense
from activation_func.relu import Activation_ReLU
from activation_func.softmax import Activation_Softmax
from loss.categorical_cross_entropy import Loss_CategoricalCrossentropy
from optimizers.adam import Optimizer_Adam
from accuracy.accuracy_categorical import Accuracy_Categorical


In [47]:
# #data preparation 
# URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
# FILE = 'fashion_mnist_images.zip'
# FOLDER = 'fashion_mnist_images'
# if not os.path.isfile(FILE):
#     print(f'Downloading {URL} and saving as {FILE}...')
#     urllib.request.urlretrieve(URL, FILE)

# print('Unzipping images...')
# with ZipFile(FILE) as zip_images:
#     zip_images.extractall(FOLDER) 

# print('Done!')

In [48]:
#load mnist dataset
def load_mnist_dataset(dataset,path):
    #list all directories and set them as label [0 to 9]
    labels = os.listdir(os.path.join(path, dataset))
    X = []
    y  = []
    
    for label in labels:
        for file in os.listdir(os.path.join(path,dataset,label)):
            image = cv2.imread(os.path.join(path, dataset, label, file),
                cv2.IMREAD_UNCHANGED)
                # And append it and a label to the lists
            X.append(image)
            y.append(label)

    return np.array(X),np.array(y).astype('uint8')

In [49]:
#create mnist dataset 
def create_data_mnist(path):
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    # And return all the data
    return X, y, X_test, y_test

In [50]:
#load dataset
X, y, X_test, y_test = create_data_mnist('fashion_mnist_images')

In [51]:
# Shuffle the training dataset
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

X.shape,y.shape

((60000, 28, 28), (60000,))

In [52]:
# Scale and reshape samples
X = (X.reshape(X.shape[0], -1).astype(np.float32) - 127.5) / 127.5
X_test = (X_test.reshape(X_test.shape[0], -1).astype(np.float32) -
127.5) / 127.5

In [53]:
model_train = Model()
X.shape,X_test.shape

((60000, 784), (10000, 784))

In [54]:
#add layers to the list 
model_train.add(Layer_Dense(X.shape[1], 64))
model_train.add(Activation_ReLU())
model_train.add(Layer_Dense(64, 64))
model_train.add(Activation_ReLU())
model_train.add(Layer_Dense(64, 10))
model_train.add(Activation_Softmax())

In [55]:
#set loss,optimizer and accuracy  
model_train.set(
loss=Loss_CategoricalCrossentropy(),
optimizer=Optimizer_Adam(decay=5e-5),
accuracy=Accuracy_Categorical()
)

In [56]:
# Finalize the model
model_train.finalize()

In [57]:
model_train.train(X, y, validation_data=(X_test, y_test),epochs=1, batch_size=128, print_every=100)

epoch1: 1
step: 0, acc: 0.156, loss: 2.303 (data_loss: 2.303, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.688, loss: 0.798 (data_loss: 0.798, reg_loss: 0.000), lr: 0.0009950248756218907
step: 200, acc: 0.742, loss: 0.636 (data_loss: 0.636, reg_loss: 0.000), lr: 0.0009900990099009901
step: 300, acc: 0.805, loss: 0.633 (data_loss: 0.633, reg_loss: 0.000), lr: 0.0009852216748768474
step: 400, acc: 0.805, loss: 0.609 (data_loss: 0.609, reg_loss: 0.000), lr: 0.000980392156862745
step: 468, acc: 0.802, loss: 0.579 (data_loss: 0.579, reg_loss: 0.000), lr: 0.0009771350400625367
training, acc: 0.720, loss: 0.755 (data_loss: 0.755, reg_loss: 0.000), lr: 0.0009771350400625367
validation, acc: 0.806, loss: 0.536


In [58]:
fashion_mnist_labels = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

In [59]:
#inference
confidences=model_train.predict(X_test[:5])
predictions=model_train.output_layer_activation.predictions(confidences)

for prediction in predictions:
    print(fashion_mnist_labels[prediction])

Sneaker
Sneaker
Sneaker
Sneaker
Ankle boot


In [60]:
import cv2
image_data = cv2.imread('tshirt.png', cv2.IMREAD_GRAYSCALE)
image_data = cv2.resize(image_data, (28, 28))
#reshape and normalize 
# Resize to the same size as Fashion MNIST images
image_data = cv2.resize(image_data, (28, 28))
# Invert image colors
image_data = 255 - image_data
# Reshape and scale pixel data
image_data = (image_data.reshape(1, -1).astype(np.float32) - 127.5) / 127.5

In [61]:
conf=model_train.predict(image_data)
predictions=model_train.output_layer_activation.predictions(conf)
prediction=fashion_mnist_labels[predictions[0]]
prediction

'T-shirt/top'